<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Фильтрация изображений с использованием дискретного преобразования Фурье

### Examples - Optics
<section class="post-meta">
By Jonas Tjemsland, Håkon Ånes, Andreas Krogen and Jon Andreas Støvneng
</section>
<section class="post-meta">
Edited by Thorvald Ballestad, Jenny Lunde, Sondre Duna Lundemo (2021)
</section>
Last edited: February 17th 2021

___

## Вступление

В этом блокноте мы обсудим двумерное преобразование Фурье (FT) и используем его для фильтрации изображений. Теория, лежащая в основе ДПФ, уже обсуждается в блокноте *[Discrete Fourier Transform and Fast Fourier Transform](https://nbviewer.jupyter.org/url/www.numfys.net/media/notebooks/discrete_fourier_transform.ipynb)*, и одномерное БПФ используется для фильтрации звуковых файлов в ноутбуке *[Simple Sound Filtering using Discrete Fourier Transform](https://nbviewer.jupyter.org/url/www.numfys.net/media/notebooks/simple_sound_filtering.ipynb)*. Если вы хотите получить более глубокое математическое и физическое понимание используемых концепций, мы отсылаем вас к этим записным книжкам или к одной из ссылок в нижней части этой записной книжки. Помните, что мы можем интерпретировать преобразование Фурье пространственной области (здесь изображения) как частотную область или частотный спектр. Для простоты мы будем использовать изображения в сером масштабе, но это можно легко обобщить.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft
from skimage import io

%matplotlib inline

# Taking the logarithm of zero in visualising the spectrum gives warnings,
# hence we disable them
warnings.filterwarnings("ignore")

## Преобразование Фурье изображения

Чтобы упростить и минимизировать реализацию кода, мы создаем функцию для визуализации изображения и соответствующего ему спектра в частотной области. Обратите внимание, что мы строим логарифм абсолютного значения, как мы делали в вышеупомянутой тетради по фильтрации звука.

In [ ]:
def visualise(image, spectrum, title="", title_img="", title_spec="", cmap="gray"):
    """Visualise an image and its corresponding frequency spectrum in one figure.

    Parameters:
        image: array-like, as in plt.imshow(). Image.
        spectrum: array-like, as in plt.imshow(). Frequency spectrum.
        title: string. Figure title. Optional.
        title_img. string. Image subtitle. Optional.
        title_spec. string. Spectrum subtitle. Optional.
        cmap: `~matplotlib.colors.Colormap`. Color map. Optional.
    """
    plt.figure(figsize=(18, 5))
    plt.subplot(121)
    plt.imshow(image, cmap=cmap)
    plt.title(title_img)
    plt.subplot(122)
    plt.imshow(np.log(abs(spectrum)), cmap=cmap)
    plt.title(title_spec)
    plt.suptitle(title)
    plt.show()

Кроме того, мы создаем две функции для вычисления преобразования Фурье и обратного преобразования Фурье изображения. Обратите внимание, что эти функции также смещают квадранты, так что низкие частоты собираются в середине спектра, а частота увеличивается с расстоянием от центра. Это соответствует большинству учебных пособий в Интернете. Однако это не является строго необходимым.

In [ ]:
def FFT(image):
    """Compute the 2D discrete Fast Fourier Transform of an image
    and shift the zero-frequency component to the center of the spectrum.

    Parameters:
        image: array-like, shape (m,n), can be complex.
    Returns: complex ndarray, shape (m,n). Frequency spectrum.
    """
    return np.fft.fftshift(np.fft.fft2(image))


def IFFT(spectrum):
    """Shift the zero-frequency component to the edges of the spectrum and
    compute the inverse 2D discrete Fast Fourier Transform of an image.

    Parameters:
        spectrum: array-like, shape (m,n), can be complex.
    Returns: complex ndarray, shape (m,n). Spacial domain.
    """
    image = np.fft.ifft2(np.fft.ifftshift(spectrum))
    return image

Давайте возьмем ПФ изображения и построим график результата.

In [ ]:
img = io.imread("https://www.numfys.net/media/notebooks/images/dog.jpg", as_gray=True)

# You may also play around with these images: (last tested March 22nd 2018)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/5/50/Albert_Einstein_%28Nobel%29.png',True)
# img = io.imread('https://upload.wikimedia.org/wikipedia/en/4/42/Richard_Feynman_Nobel.jpg',True)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/c/cf/Dirac_4.jpg',True)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/7/73/Stephen_Hawking_in_Cambridge.jpg',True)

spec = FFT(img)
visualise(img, spec, title_img="Input image", title_spec="Spectrum")

## Фильтрация

Существует бесконечное количество различных фильтров, которые можно использовать в частотной области для фильтрации исходного изображения. Мы реализуем фильтры, которые отфильтровывают низкие и высокие частоты в фильтре высоких и низких частот соответственно. В обоих случаях используется идеальный фильтр и фильтр Гаусса.

Предположим, что рассматриваемое изображение имеет $n$ пикселей по горизонтали ($x$-направление) и $m$ пикселей по вертикали ($y$-направление). Пусть $x=0,1,...,n-1$ и $y=0,1,..,m-1$ определяют квадратную решетку $x \times y$. Теперь обозначим частотный спектр как $\mathcal F$, идеальные фильтры как $I_\text{lowpass}$ и $I_\text{highpass}$, а гауссовы фильтры как $G_\text{lowpass}$ и $G_\text{highpass}$, каждый из которых определен на решетке $x \times y$. Мы определяем эти фильтры таким образом, что отфильтрованный спектр задается произведением *Адамара* (поэлементное матричное произведение) $\mathcal F\circ I$ и $\mathcal F\circ G$.

$r=\sqrt{(x-n/2)^2+(y-m/2)^2}\geq0$ - расстояние от центра спектра. Рассматриваемые нами фильтры зависят только от одного параметра $\xi\in \mathbb{R}^+$. В этой нотации идеальные фильтры задаются

$$
I_\text{lowpass}=\begin{cases}
1,& \text{for } r\leq\xi,\\
0,& \text{for } r>\xi,
\end{cases}
\quad \text{and} \quad
I_\text{highpass}=\begin{cases}
1,& \text{for } r\geq\xi,\\
0,& \text{for } r<\xi,
\end{cases}
$$

и гауссовы фильтры задаются

$$
G_\text{lowpass}=e^{-\xi r^2}
\quad \text{and} \quad
G_\text{highpass}=1-e^{-\xi r^2}.
$$

In [ ]:
def filter_spectrum(spectrum, filter_type=None, val=50):
    """Фильтрует спектр изображения с помощью одного из фильтров, определенных в приведенном выше тексте.

    Parameters:
        spectrum:    array-like, shape (m, n), can be complex. Spectrum being filtered.
        filter_type: string. Filter name,
                     {'lowpass', 'highpass', 'gaussian_highpass', 'gaussian_lowpass'}.
        val:         float. Filter parameter.
    Returns: complex ndarray, shape (m, n). Filtered spectrum.
    """
    n, m = np.shape(spectrum)
    y, x = np.meshgrid(np.arange(1, m + 1), np.arange(1, n + 1))
    R2 = (x - n / 2) ** 2 + (y - m / 2) ** 2
    if filter_type == "lowpass":
        return spectrum * (R2 <= val ** 2)
    elif filter_type == "highpass":
        return spectrum * (R2 >= val ** 2)
    elif filter_type == "gaussian_highpass":
        return spectrum * (1 - np.exp(-val * R2))
    elif filter_type == "gaussian_lowpass":
        return spectrum * np.exp(-val * R2)
    elif filter_type != None:
        raise ValueError("%s is not a valid filter!" % (filter_type))
    return spectrum

Теперь мы используем эти фильтры на частотном спектре и обратные преобразования Фурье для получения отфильтрованных изображений. Обратите внимание, что изображения, отфильтрованные с помощью гауссовых фильтров, несколько более гладкие, чем изображения, отфильтрованные с помощью идеальных фильтров.


Поиграйте с фрагментами кода!

### Низкочастотный фильтр

In [ ]:
filtered_spec = filter_spectrum(spec, "lowpass", 40)
filtered_img = np.real(IFFT(filtered_spec))
visualise(filtered_img, filtered_spec, title="Ideal lowpass filtering")

In [ ]:
filtered_spec = filter_spectrum(spec, "gaussian_lowpass", 0.001)
filtered_img = np.real(IFFT(filtered_spec))
visualise(filtered_img, filtered_spec, title="Gaussian lowpass filtering")

### Высокочастотный

In [ ]:
filtered_spec = filter_spectrum(spec, "highpass", 10)
filtered_img = np.real(IFFT(filtered_spec))
visualise(filtered_img, filtered_spec, title="Ideal highpass filtering")

In [ ]:
filtered_spec = filter_spectrum(spec, "gaussian_highpass", 0.01)
filtered_img = np.real(IFFT(filtered_spec))
visualise(filtered_img, filtered_spec, title="Gaussian highpass filtering")

## Отфильтровывание специфических паттернов и шума

Принципы, представленные в этой записной книжке, могут быть легко применены при фильтрации изображений, точно так же, как одномерные преобразования Фурье использовались для фильтрации звуковых файлов в вышеупомянутой записной книжке по фильтрации звука. В двух следующих примерах мы включаем изображение с некоторыми шумовыми паттернами. Эти изображения преобразуются Фурье в частотную область. Любые аномалии удаляются, и мы получаем отфильтрованное изображение, преобразуясь обратно в пространственную область.

### Пример 1

In [ ]:
img = io.imread("https://www.numfys.net/media/notebooks/images/dog_noise1.jpg", True)
spec = FFT(img)
visualise(img, spec, title_img="Noisy image", title_spec="Unfiltered spectrum")

In [ ]:
# Create a filter which filters out the circular anomaly
n, m = np.shape(img)
y, x = np.meshgrid(np.arange(m), np.arange(n))
R2 = (x - n / 2) ** 2 + (y - m / 2) ** 2
filtered_spec = spec * (R2 <= 100 ** 2) + spec * (R2 >= 130 ** 2)
filtered_img = np.real(IFFT(filtered_spec))

# Visualise result
visualise(
    filtered_img,
    filtered_spec,
    title_img="Filtered image",
    title_spec="Filtered spectrum",
)

### Пример 2

In [ ]:
img = io.imread("https://www.numfys.net/media/notebooks/images/dog_noise2.jpg", True)
spec = FFT(img)
visualise(img, spec, title_img="Noisy image", title_spec="Unfiltered spectrum")

In [ ]:
# Create a filter which filters out the anomaly on the axes (+)
n, m = np.shape(img)
y, x = np.meshgrid(np.arange(m), np.arange(n))
filtered_spec = spec * (x <= n / 2 - 1) + spec * (x >= n / 2 + 1)
filtered_spec = filtered_spec * (y <= m / 2 - 1) + filtered_spec * (y >= m / 2 + 1)
filtered_img = np.real(IFFT(filtered_spec))

# Visualise result
visualise(
    filtered_img,
    filtered_spec,
    title_img="Filtered image",
    title_spec="Filtered spectrum",
)

# Реализация на основе классов

Мы в NumFys недавно написали руководство по [объектно-ориентированному программированию(ООП) на Python](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/classes_in_Python.ipynb).
ООП - это мощная парадигма, которая, безусловно, имеет свое место в численном программировании.
В нашей записной книжке по [нейронным сетям](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ML_from_scratch_tekst.ipynb) мы использовали структуру на основе классов, чтобы поддерживать чистоту базы кода.
Однако этот блокнот довольно сложен и демонстрирует только некоторые особенности ООП.
Мы надеемся, что наблюдение за фильтрацией изображений, реализованной как в парадигме, основанной на функциях, так и с использованием подхода, основанного на классах, поможет вам в разработке мышления для разработки ООП.

Сначала мы создаем класс `Filter`, который будет служить нашим базовым классом.
Никогда не создавалось никаких экземпляров этого класса;
для тех, кто знаком с ООП на других языках, мы рассматриваем этот класс как абстрактный или виртуальный класс.
Все фильтры, реализованные выше, имеют то общее, что они выполняют некоторое преобразование спектра, преобразование фурье, изображения.
Поэтому мы создаем класс `SpectrumFilter`, который является подклассом `Filter`, и реализуем каждый из фильтров спектра как подкласс.
Причина добавления `SpectrumFilter` вместо обычного использования `Filter` заключается в том, что мы в демонстрационных целях хотим добавить еще несколько фильтров, которые не основаны на спектре.

In [ ]:
class Filter:
    """Abstract class for a filter.

    Subclasses must implement the `render` method."""

    def __init__(self, val=1):
        # Val is some parameter used by the filter.
        # Its specific meaning varies between filters.
        self.val = val

    def render(self, image):
        """Applies the filter to `image`."""
        return image

    def visualize(
        self, title="", title_img="", title_spec="", cmap="gray", figsize=(18, 5)
    ):
        assert hasattr(self, "image"), "The filter has not rendered any image!"
        if not hasattr(self, "spectrum"):
            self.spectrum = FFT(self.image)

        plt.figure(figsize=figsize)
        plt.subplot(121)
        plt.imshow(self.image, cmap=cmap, vmin=0, vmax=1)
        plt.title(title_img)
        plt.xticks([])
        plt.yticks([])

        plt.subplot(122)
        plt.imshow(np.log(abs(self.spectrum)), cmap=cmap)
        plt.title(title_spec)
        plt.xticks([])
        plt.yticks([])

        plt.suptitle(title)
        plt.show()

    def render_and_visualize(self, image, **kwargs):
        """Convenience function for rendering and visualizing."""
        self.render(image)
        self.visualize(**kwargs)
        return self.image

    def __call__(self, img):
        return self.render_and_visualize(img)


class SpectrumFilter(Filter):
    """Abstract class for a spectrum filter.
    A spectrum filter does some transformation to the spectrum (FT) of the image."""

    def spectrum_criterion(self, R2, val):
        pass

    def render(self, image):
        spectrum = FFT(image)
        n, m = np.shape(spectrum)
        y, x = np.meshgrid(np.arange(1, m + 1), np.arange(1, n + 1))
        R2 = (x - n / 2) ** 2 + (y - m / 2) ** 2
        spectrum *= self.spectrum_criterion(R2)

        # Store spectrum and image
        self.spectrum = spectrum
        self.image = np.real(IFFT(spectrum))
        # Rescale image to be within 0 1
        a, b = np.min(self.image), np.max(self.image)
        self.image = (self.image - a) / (b - a)
        return self.image


class HighpassSpectrumFilter(SpectrumFilter):
    def spectrum_criterion(self, R2):
        return R2 >= self.val ** 2


class LowpassSpectrumFilter(SpectrumFilter):
    def spectrum_criterion(self, R2):
        return R2 <= self.val ** 2


class GaussianHighpassSpectrumFilter(SpectrumFilter):
    def spectrum_criterion(self, R2):
        return 1 - np.exp(-self.val * R2)


class GaussianLowpassSpectrumFilter(SpectrumFilter):
    def spectrum_criterion(self, R2):
        return np.exp(-self.val * R2)


class CircularSpectrumFilter(SpectrumFilter):
    def __init__(self, r1, r2, include=False):
        self.r1, self.r2 = sorted([r1, r2])
        self.include = include

    def spectrum_criterion(self, R2):
        # r1 and r2 defines a disc, where r1 < r2.
        on_disc = np.logical_and(R2 >= self.r1 ** 2, R2 <= self.r2 ** 2)
        # If include is True, keep the disc and filter out everything else.
        # Else, filter out the disc.
        if self.include:
            return on_disc
        else:
            return np.logical_not(on_disc)

In [ ]:
# You may also play around with these images: (last tested January 22nd 2021)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/5/50/Albert_Einstein_%28Nobel%29.png',True)
img = io.imread(
    "https://upload.wikimedia.org/wikipedia/en/4/42/Richard_Feynman_Nobel.jpg", True
)
# img = io.imread('https://www.numfys.net/media/notebooks/images/dog.jpg', as_gray=True)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/c/cf/Dirac_4.jpg',True)
# img = io.imread('https://upload.wikimedia.org/wikipedia/commons/7/73/Stephen_Hawking_in_Cambridge.jpg',True)

In [ ]:
ghp_filter = GaussianHighpassSpectrumFilter(0.005)
hp_filter = HighpassSpectrumFilter(10)

ghp_filter.render_and_visualize(img)
hp_filter.render_and_visualize(img);

In [ ]:
glp_filter = GaussianLowpassSpectrumFilter(0.005)
lp_filter = LowpassSpectrumFilter(10)

glp_filter.render_and_visualize(img)
# We use the __call__ method.
lp_filter(img);

In [ ]:
circular_filter = CircularSpectrumFilter(100, 130)
img_w_noise = io.imread(
    "https://www.numfys.net/media/notebooks/images/dog_noise1.jpg", True
)
circular_filter(img_w_noise);

Одним из преимуществ нашего подхода, основанного на классе, является то, что расширение нашего оборудования является простым.
Мы создаем два новых типа фильтров, чтобы продемонстрировать расширяемость нашей системы.

In [ ]:
class ContrastFilter(Filter):
    """A very naive implmentation of a contrast filter."""

    def render(self, image):
        x = self.val
        self.image = np.clip(image * x + 0.5 * (1 - x), 0, 1)
        return self.image


class CombineFilter(Filter):
    """A filter which executes an arbitrary amount of other filters in sequence."""

    def __init__(self, filter_1, filter_2, *args):
        self.filters = [filter_1, filter_2, *args]

    def render(self, image):
        self.image = image
        for filter in self.filters:
            self.image = filter.render(self.image)
        return self.image

    def render_and_visualize_step(self, image):
        self.image = image
        self.spectrum = FFT(image)
        self.visualize()
        for filter in self.filters:
            print(filter)
            self.image = filter.render_and_visualize(self.image)
        return self.image
    
    def __add__(self, other_filter):
        self.filters.append(other_filter)
        return self


def __add__(self, other_filter):
    return CombineFilter(self, other_filter)


setattr(Filter, "__add__", __add__)

In [ ]:
cf = ContrastFilter(1.5)
cf(img);

In [ ]:
blur_contrast = CombineFilter(glp_filter, cf)
blur_contrast_2 = glp_filter + cf  # The same as above, but using overridden operators.

blur_contrast(img)
blur_contrast_2(img);

Наконец, посмотрите, понимаете ли вы, основываясь на следующем коде, почему у `CombineFilter` иной `__add__` метод, чем у остальных фильтров.

In [ ]:
(glp_filter + cf + circular_filter)(img)
# Uncomment to see each step performed.
# (cf + circular_filter + glp_filter).render_and_visualize_step(img)
(cf + circular_filter + glp_filter)(img);

## Дальнейшая работа и улучшения

+ Сделайте то же в цвете.

### Further reading

All of them acquired 10-28-2016.

* Robert Fisher, Simon Perkins, Ashley Walker and Erik Wolfart: Fourier Transform, http://homepages.inf.ed.ac.uk/rbf/HIPR2/fourier.htm, 2000.
* Paul Bourke: Image Filtering in the Frequency Domain, http://paulbourke.net/miscellaneous/imagefilter/, 1998.
* Steven Lehar: An Intuitive Explanation of Fourier Theory, http://cns-alumni.bu.edu/~slehar/fourier/fourier.html.

Multi-dimensional image processing using scipy: https://docs.scipy.org/doc/scipy-0.18.1/reference/ndimage.html